#   Homework 3
## Sentiment analysis using Neural Networks

Total: 50 Points


In this homework we will perform sentiment analysis using a few simple Neural Network based architectures.
For this problem we use the IMDB Large Movie Review Dataset. The dataset contains 25,000 highly polar movie reviews for both train and test dataset, each with 12,500 positive (greater than equal to 7/10 rating) and 12,500 negative reviews(less than equal to 4/10 rating). 

Use "https://keras.io/" for keras documentation. Please use Python 3. GPU is not required but it will help improve the training speed for each problem.

Please save the notebook with your cell outputs. You will not be graded if your outputs are not present below the homework cell. Also note your outputs will be unique since you will be using your the last numbers of your uni as your random seed (In the third cell). Make sure you submit this iPython file, with the saved outputs. The submission format must be 'hw3/hw3.ipynb'. You will not submit any other files. If you do save your model weights, you will not submit them. You will however, make sure your model weights do get saved in the 'weights' folder and can be retrieved from there as well.

Please fill your details below.



Name: Xiaohui Guo

Uni: xg2225

Email: xg2225@columbia.edu


In [1]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np
import h5py
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:


#we retrieve train and test file names

train_dir = "./aclImdb/train/"
test_dir = "./aclImdb/test/"
tr_review = [re_filename for re_filename in listdir(train_dir)]
te_review = [re_filename for re_filename in listdir(test_dir)]

#we initialize the train and test arrays

tr_X = []
tr_Y = []
te_X = []
te_Y = []

#we arrange the reviews into the train and test arrays 

for review_file in tr_review:
    f_review = open(train_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    tr_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        tr_Y.append(1)
    else:
        tr_Y.append(0)
        
for review_file in te_review:
    f_review = open(test_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    te_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        te_Y.append(1)
    else:
        te_Y.append(0)
        

We will now create the validation set from the train set

use the last 4 numbers of your uni for the seed value seed to ensure all answers remain unique.

In [3]:
#replace 2 (SEED) with the last 4 numbers of your Uni
#Uni: 
SEED = 2225
seed_counter = 0
while(1):

    shuffle_combine = list(zip(tr_X, tr_Y))
    random.seed(SEED+seed_counter)
    seed_counter+=1
    random.shuffle(shuffle_combine)

    tr_X, tr_Y = zip(*shuffle_combine)

    val_X = tr_X[:5000]
    val_Y = tr_Y[:5000]

    counter = 0
    for label in val_Y:
        counter+=label

    print (counter)
    print (seed_counter)
    if(counter>2400 and counter <2600):
        tr_X = tr_X[5000:]
        tr_Y = tr_Y[5000:]
        break

2517
1


In [4]:


print("Length of Train review set : " + str(len(tr_X)))
print("Length of Train label set : " + str(len(tr_Y)))
print("Length of Validation review set : " + str(len(val_X)))
print("Length of Validation label set : " + str(len(val_Y)))
print("Length of Test review set : " + str(len(te_X)))
print("Length of Test label set : " + str(len(te_Y)))
print("*****************************************")
print("Some sample Reviews Train sets and their labels")
print(tr_X[0][:150])
print(tr_Y[0])
print(tr_X[1][:150])
print(tr_Y[1])
print(tr_X[2][:150])
print(tr_Y[2])
print(tr_X[3][:150])
print(tr_Y[3])
print(tr_X[4][:150])
print(tr_Y[4])

Length of Train review set : 20000
Length of Train label set : 20000
Length of Validation review set : 5000
Length of Validation label set : 5000
Length of Test review set : 25000
Length of Test label set : 25000
*****************************************
Some sample Reviews Train sets and their labels
Having to have someone hold your hand whenever walk up or down stairs? Having others taste your food before you eat it? Facing an over-bearing mother?
1
This is a hard-boiled Warner Brothers film starring a very young Barbara Stanwyck. A consummate master at portraying Machiavellian cool, a technique s
1
Its hard to make heads or tails of this film. Unless you're well oiled and in the mood to mock, don't view Santa Claus. It mixes Santa, Satan, Merlin,
0
The best of the seven Sam Fuller movies that I've seen (including Park Row, Run of the Arrow, Verboten!, Shock Corridor, The Naked Kiss, The Big Red O
1
This is a great show, and will make you cry, this group people really loved each othe

In [5]:
#we collect all the reviews from train validation and test set to generate 
texts = []
texts += tr_X 
texts += te_X 
texts += val_X
len(texts)



#we clip the sentence length to first 250 words. 
MAX_SEQUENCE_LENGTH = 250

#length of vocab, Tokenizer will only use vocab_len most common words
vocab_len = 25000

#we tokenize the texts and convert all the words to tokens
tokenizer = Tokenizer(num_words=vocab_len)
tokenizer.fit_on_texts(texts)

token_tr_X = tokenizer.texts_to_sequences(tr_X)
token_te_X = tokenizer.texts_to_sequences(te_X)
token_val_X = tokenizer.texts_to_sequences(val_X)

#to ensure all reviews have the same length, we pad the smaller reviews with 0, 
#and cut the larger reviews to a max length 
#(we clip from the top, as the end of the reviews generally have a conclusion which provides better features)
x_train = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
x_test = sequence.pad_sequences(token_te_X, maxlen=MAX_SEQUENCE_LENGTH)
x_val = sequence.pad_sequences(token_val_X, maxlen=MAX_SEQUENCE_LENGTH)


#changes the labels to one-hot encoding
y_train = np_utils.to_categorical(tr_Y)
y_test = np_utils.to_categorical(te_Y)
y_val = np_utils.to_categorical(val_Y)


In [6]:
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('X_val shape:', x_val.shape)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_val shape:', y_val.shape)


print("*****************************************")
print("Tokenized Reviews Train sets and their labels")
print(x_train[0][:20])
print(y_train[0])
print()
print(x_train[1][:20])
print(y_train[1])
print()
print(x_train[2][:20])
print(y_train[2])
print()
print(x_train[3][:20])
print(y_train[3])
print()
print(x_train[4][:20])
print(y_train[4])
print()

X_train shape: (20000, 250)
X_test shape: (25000, 250)
X_val shape: (5000, 250)
y_train shape: (20000, 2)
y_test shape: (25000, 2)
y_val shape: (5000, 2)
*****************************************
Tokenized Reviews Train sets and their labels
[ 770 8370    6  898    2  162   32 1598    8    1   19    7    7  259    1
   17  124 2432    5 1861]
[ 0.  1.]

[    3   249  7951  2991  1177    19  1212     3    52   186  2164  3337
     3 11648  1199    30  2339 24417   593     3]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[    1  1582  1285  6594    97    12   198   107   585  1274  3118   495
     4     1  7364  1472 13963     1  1286  2589]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  1.]



********************************************

As you can see the reviews have now been transformed into indices to tokenized vocabulary and the labels have been converted to one-hot encoding. We can now go ahead and feed these sequences to Neural Network Models.

********************************************

# Part A

Building your first model (5 Points)

Construct this sequential model using Keras :

![title](img/model1.jpg)

In [7]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH ,trainable = True))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))


## compille it here according to instructions
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               6400200   
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 9,600,602
Trainable params: 9,600,602
Non-trainable params: 0
___________________________________________________

In [8]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 44s 2ms/step - loss: 0.4140 - acc: 0.7913 - val_loss: 0.3156 - val_acc: 0.8684
Epoch 2/4
20000/20000 [==============================] - 41s 2ms/step - loss: 0.0619 - acc: 0.9800 - val_loss: 0.4352 - val_acc: 0.8494
Epoch 3/4
20000/20000 [==============================] - 40s 2ms/step - loss: 0.0037 - acc: 0.9994 - val_loss: 0.5431 - val_acc: 0.8632
Epoch 4/4
20000/20000 [==============================] - 38s 2ms/step - loss: 2.5132e-04 - acc: 1.0000 - val_loss: 0.6152 - val_acc: 0.8624


# Part B

Stacking Fully Connected Layers (5 points)

Construct this sequential model using Keras :

![title](img/model2.jpg)

In [9]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH ,trainable = True))
model.add(Flatten())
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))



## compille it here according to instructions
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               6400200   
_________________________________________________________________
activation_3 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
activation_4 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402   

In [10]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 35s 2ms/step - loss: 0.4021 - acc: 0.8008 - val_loss: 0.3089 - val_acc: 0.8706
Epoch 2/4
20000/20000 [==============================] - 35s 2ms/step - loss: 0.0486 - acc: 0.9842 - val_loss: 0.4224 - val_acc: 0.8484
Epoch 3/4
20000/20000 [==============================] - 35s 2ms/step - loss: 0.0058 - acc: 0.9983 - val_loss: 0.6826 - val_acc: 0.8500
Epoch 4/4
20000/20000 [==============================] - 35s 2ms/step - loss: 0.0135 - acc: 0.9956 - val_loss: 0.7160 - val_acc: 0.8430


# Part C

Using LSTMS based networks(5 Points) 

Construct this sequential model using Keras :

![title](img/model3.jpg)

In [11]:
print('Build model...')

## implement model here

model = Sequential()

model.add(Embedding(vocab_len, 128, input_length=MAX_SEQUENCE_LENGTH ,trainable = True))
model.add(LSTM(128))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))


## compille it here according to instructions

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_6 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_7 (Activation)    (None, 2)                 0         
Total params: 3,348,354
Trainable params: 3,348,354
Non-trainable params: 0
___________________________________________________

In [12]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 234s 12ms/step - loss: 0.4383 - acc: 0.7915 - val_loss: 0.3428 - val_acc: 0.8594
Epoch 2/5
20000/20000 [==============================] - 227s 11ms/step - loss: 0.2059 - acc: 0.9224 - val_loss: 0.3708 - val_acc: 0.8612
Epoch 3/5
20000/20000 [==============================] - 229s 11ms/step - loss: 0.1093 - acc: 0.9608 - val_loss: 0.4354 - val_acc: 0.8590
Epoch 4/5
20000/20000 [==============================] - 233s 12ms/step - loss: 0.0718 - acc: 0.9757 - val_loss: 0.5474 - val_acc: 0.8618
Epoch 5/5
20000/20000 [==============================] - 236s 12ms/step - loss: 0.0433 - acc: 0.9860 - val_loss: 0.6439 - val_acc: 0.8614


# Part D

Adding Pretrained Word Embeddings(10 Points)

Construct this sequential model using Keras :

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model4.jpg)

In [7]:
import codecs

#dimension of Glove Embeddings.
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

#load glove embeddings
gembeddings_index = {}
with codecs.open('glove.42B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

# nb_words contains the total length of vocab
nb_words = len(word_index) +1

#get glove embeddings for each word in tokenizer.
#g_word_embedding_matrix holds the embeddings dictionary
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
#total words in the tokenizer not in Embedding matrix
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))



Found 124252 unique tokens
G Word embeddings: 1917494
G Null word embeddings: 35772


In [14]:
print('Build model...')

## implement model here

## to use the glove embeddings, your embedding layer would take the vocab size as input dimension, 
## Glove embedding dimension as the output dimsion
## and you will provide the  embedding dictionary as the 'weights' parameter (!important) to the embedding layer.



model = Sequential()


model.add(Embedding(nb_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,
                    trainable = False,
                    weights = [g_word_embedding_matrix]))
model.add(LSTM(128,recurrent_dropout = 0.2))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))





## compille it here according to instructions

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_9 (Activation)    (None, 2)                 0     

In [15]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 179s 9ms/step - loss: 0.4562 - acc: 0.7843 - val_loss: 0.3375 - val_acc: 0.8566
Epoch 2/5
20000/20000 [==============================] - 177s 9ms/step - loss: 0.2937 - acc: 0.8809 - val_loss: 0.3251 - val_acc: 0.8708
Epoch 3/5
20000/20000 [==============================] - 180s 9ms/step - loss: 0.2519 - acc: 0.8977 - val_loss: 0.3051 - val_acc: 0.8712
Epoch 4/5
20000/20000 [==============================] - 181s 9ms/step - loss: 0.2158 - acc: 0.9146 - val_loss: 0.2994 - val_acc: 0.8834
Epoch 5/5
20000/20000 [==============================] - 180s 9ms/step - loss: 0.1789 - acc: 0.9306 - val_loss: 0.3033 - val_acc: 0.8832


# Dont attempt this

Stacking LSTM layers

Unfortunately it takes very long to train, be aware we can stack LTMSs over each other like this.
This requires bottom LSTM to return a sequences instead instead of single vector, which becomes input for the top LSTM.


![title](img/model5.jpg)

# Part E

Using Convolutional Networks (10 points)

Construct the model, shown below. Use the same loss functions and optimizers as before

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model6.jpg)

In [16]:
print('Build model...')

## implement model here

model = Sequential()

model.add(Embedding(nb_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,
                    trainable = False,
                    weights = [g_word_embedding_matrix]))

model.add(Convolution1D(filters = 128, kernel_size = 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Convolution1D(filters = 64, kernel_size = 3))        
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Convolution1D(filters = 32, kernel_size = 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(2))
model.add(Activation('softmax'))




## compille it here according to instructions
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])


model.summary()

print("Model Built")


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_10 (Activation)   (None, 248, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_3 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_11 (Activation)   (None, 246, 64)           0     

In [17]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

# Accuracy: 89.32%

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 50s 3ms/step - loss: 0.4481 - acc: 0.7749 - val_loss: 0.3648 - val_acc: 0.8398
Epoch 2/5
20000/20000 [==============================] - 48s 2ms/step - loss: 0.3005 - acc: 0.8758 - val_loss: 0.2903 - val_acc: 0.8784
Epoch 3/5
20000/20000 [==============================] - 48s 2ms/step - loss: 0.2607 - acc: 0.8934 - val_loss: 0.3458 - val_acc: 0.8568
Epoch 4/5
20000/20000 [==============================] - 49s 2ms/step - loss: 0.2292 - acc: 0.9082 - val_loss: 0.2967 - val_acc: 0.8814
Epoch 5/5
20000/20000 [==============================] - 48s 2ms/step - loss: 0.1972 - acc: 0.9207 - val_loss: 0.3159 - val_acc: 0.8862


# Part F

Model constructed : (5 points)

Test Accuracy Over 87.5%: (5 Points)

Bonus: Min(10, Square of (test_score - 88%))

Create your best model, use Validation score to judge your best model and check accuracy on test set


In [19]:
print('Build model...')

## implement model here



model = Sequential()


model.add(Embedding(nb_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,
                    trainable = False,
                    weights = [g_word_embedding_matrix]))
model.add(LSTM(128,recurrent_dropout = 0.2))
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))





## compille it here according to instructions

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
activation_6 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_7 (Activation)    (None, 2)                 0     

In [ ]:
# model1: Question A   Accuracy: 86.76%    Run time OK
# model2: Question B   Accuracy: 87.10%    Run time OK
# model3: Question C   Accuracy: 84.89%    Run a long time
# model4_D: Question D Accuracy: 89.62%
# model5: Question E   Accuracy: 89.17%
# model6: D modify dropout rate 0.3  Accuracy: 89.46%
# model7 D modify dropout rate 0.1  recurrent_dropout = 0.1   89.53%
# model8 D modify dropout rate 0.2  recurrent_dropout = 0.3   Accuracy: 89.61%
# model 10 Add Activation Dense 89.24%

You can keep saving models with different names in model_name, 

so you can retrieve their weights again for testing, you dont have to retrain 
(You would have to initialize the model definition again).

In [20]:
wt_dir = "./weights/"
model_name = 'model10'
early_stopping =EarlyStopping(monitor='val_acc', patience=2)
bst_model_path = wt_dir + model_name + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True)

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=7,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True,
         callbacks=[early_stopping, model_checkpoint])




Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/7
20000/20000 [==============================] - 225s 11ms/step - loss: 0.4478 - acc: 0.7907 - val_loss: 0.3697 - val_acc: 0.8420
Epoch 2/7
20000/20000 [==============================] - 216s 11ms/step - loss: 0.3003 - acc: 0.8750 - val_loss: 0.3060 - val_acc: 0.8702
Epoch 3/7
20000/20000 [==============================] - 207s 10ms/step - loss: 0.2489 - acc: 0.8992 - val_loss: 0.2937 - val_acc: 0.8804
Epoch 4/7
20000/20000 [==============================] - 205s 10ms/step - loss: 0.2139 - acc: 0.9144 - val_loss: 0.2925 - val_acc: 0.8808
Epoch 5/7
20000/20000 [==============================] - 209s 10ms/step - loss: 0.1688 - acc: 0.9347 - val_loss: 0.3040 - val_acc: 0.8862
Epoch 6/7
20000/20000 [==============================] - 220s 11ms/step - loss: 0.1286 - acc: 0.9511 - val_loss: 0.3751 - val_acc: 0.8720
Epoch 7/7
20000/20000 [==============================] - 207s 10ms/step - loss: 0.0861 - acc: 0.9681 - val_loss: 0

If you plan on using Ensemble averaging, feel free to edit the code below or add multiple models.

Make sure they get saved and can be retrieved when executing serially.

In [21]:
model_dir = "./models/"
saved_model_path = model_dir + model_name + '.json'

# serialize model to JSON
model_json = model.to_json()
with open(saved_model_path, "w") as json_file:
    json_file.write(model_json)
print("Saved model to disk")

Saved model to disk


In [25]:
# load json and create model
json_file = open('./models/model4_D.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
print("Loaded model from disk")


## compille it here according to instructions
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

Loaded model from disk


In [26]:
model.load_weights('./weights/model4_D.h5')
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 82s 3ms/step
Accuracy: 89.62%


# Part G

Explain how Dense, LSTM and Convolution Layers work.

Explain Relu, Dropout, and Softmax work.

Analyze the architectures you constructed, with the accuracies you achieved and the training time it took. 

What are some insights you gained with these experiments? 

(5 Points)


### Answer:

#### Dense Layer

The Dense layer is a fully connected layer. All the neurons in a layer are connected to those in the next layer. The Dense Layer represents a matrix vector multiplication. The values in the matrix are the trainable parameters which get updated during the backpropagation. The dense layer is used to change the dimensions of the vector. From the point of view of mathematic, the dense layer applies a rotation, scaling, translation transform to the vector.

#### LSTM Layer

LSTM is short for Long Short Term Memeory Layer. The layer is applied for simple recurrent neural network which can be used as a building component or block of hidden layers for an finally bigger recurrent neural network. 

#### Convolution Layer

Convolution layer creates a convolution kernel that is convolved with the layer input over a single spatial dimension to produce a tensor of outputs. 

#### Relu

Relu function is f(x) = max(0,x). It is applied element-wise to the output of some other function. Relu improve neural networks's training speed. The gradient computation is simple and the computational step is easy. All the negative elements are set to 0. There is no exponentials, multiplication or division operations. 

#### Dropout

Using dropout can randomly deactivate certain neurons in a layer with a certain probability p from a Bernoulli distribution. The neural network won't be able to rely on particular activations in a given feed forward pass dring trainin. The neural network will learn different, redundant representations. What's more, the network can't reply on the particular neurons and the combination of these neurons to be present. The dropout layer doesn't have any trainable parameters.The dropout drops conections of neurons from the dense layer to prevent overfitting. The training will be fater when use the dropout layer.

#### Softmax

A softmax layer is to convert any vector of real numbers into a vector of probabilities. 

#### The Architecture I constructed

The Accuracy I achieved is 89.62%.

The running time is 1489 seconds.

First of all, add embedding layer, set the output dimension to be 300. Then add a LSTM layer, set the recurrent dropout to be 0.2. Set the dimension to be 128. Then set a dropout layer,set the rate to be 0.2. The training will be faster after adding this layer. Then add a dense layer, with the output dimension to be 128.Then add a Relu layer. Then add another dense layer and set the output dimension to be 2. In the end, add a softmax layer to convert vector of real numbers into a vector of probabilities.

####  Insights

Since the models are so large and it took some time to train the model that it is highly efficiency to use h5py to store models and load models. When design the architecture, change parameters and find the model that could have the highest accuracy when testing on test data, and also consider about the reducing running time.